# data validation 
- datatypes 
- data drift 
- same schema
- validate column


In [6]:
# constants 
""" 
Data Validation related constants 
"""
DATA_VALIDATION_DIR_NAME: str = 'data_validation'
DATA_VALIDATION_VALID_DIR: str = 'validated'
DATA_VALIDATION_INVALID_DIR: str = 'invalid'
DATA_VALIDATION_DRIFT_REPORT_DIR: str = 'drift_report'
DATA_VALIDATION_DRIFT_REPRT_FILE_NAME: str = 'report.yaml'

In [7]:
# config 
from NetworkSecurity.constants import training_pipeline
from NetworkSecurity.config.configuration import TrainingPipelineConfig
import os 

class DataValidationConfig: 
    def __init__(self,training_pipeline_config:TrainingPipelineConfig):
        self.data_validation_dir: str = os.path.join(training_pipeline_config.artifact_dir,training_pipeline.DATA_VALIDATION_DIR_NAME)
        self.valid_data_dir: str = os.path.join(self.data_validation_dir,training_pipeline.DATA_VALIDATION_VALID_DIR)
        self.invalid_data_dir: str = os.path.join(self.data_validation_dir, training_pipeline.DATA_VALIDATION_INVALID_DIR)
        self.valid_train_file_path: str =os.path.join(self.valid_data_dir, training_pipeline.TRAIN_FILE_NAME)
        self.valid_test_file_path: str = os.path.join(self.valid_data_dir, training_pipeline.TEST_FILE_NAME)
        self.invalid_train_file_path: str = os.path.join(self.invalid_data_dir, training_pipeline.TRAIN_FILE_NAME)
        self.invalid_test_file_path: str = os.path.join(self.invalid_data_dir, training_pipeline.TEST_FILE_NAME)
        self.drift_report_file_path: str = os.path.join(self.data_validation_dir, 
                                                        training_pipeline.DATA_VALIDATION_DRIFT_REPORT_DIR, 
                                                        training_pipeline.DATA_VALIDATION_DRIFT_REPRT_FILE_NAME
                                                        )

In [8]:
# create schema.yaml file

In [ ]:
# component 
from NetworkSecurity.entity.artifact_entity import DataIngestionArtifact, DataValidationAritifact
# from NetworkSecurity.entity.config_entity import D
from NetworkSecurity.exception.exception import NetworkSecurityException
from NetworkSecurity.logging.logger import logging
from scipy.stats import ks_2samp # check 2 samples for data drift 
from NetworkSecurity.constants.training_pipeline import SCHEMA_FILE_PATH
from NetworkSecurity.utils.main_utils.utils import read_yaml_file
import pandas as pd 
import os, sys 

class DataValidation:
    def __init__(self, data_ingestion_artifact: DataIngestionArtifact,
                 data_validation_config: DataValidationConfig):
        try:
            self.data_ingestion_artifact=data_ingestion_artifact
            self.data_validation_config=data_validation_config
            self._schema_config=read_yaml_file(SCHEMA_FILE_PATH)
        except Exception as e:
            raise NetworkSecurityException(e,sys)
        
    @staticmethod
    def read_data(file_path)->pd.DataFrame:
        try:
            return pd.read_csv(file_path)
        except Exception as e:
            raise NetworkSecurityException(e,sys)

    def validate_numbers_of_columns(self,dataframe:pd.DataFrame)->bool:
        try:
            number_of_columns=len(self._schema_config) 
            logging.info(f'Required number of columns {number_of_columns}')
            logging.info(f'Data frame has columns: {len(dataframe.columns)}')
            if len(dataframe.columns)==number_of_columns:
                return True 
            return False 
        except Exception as e:
            raise NetworkSecurityException(e,sys)

    def detect_dataset_drift(self,base_df,current_df,threshold=0.05,)
    def initiate_data_validation(self)-> DataValidationAritifact:
        try:
            train_file_path=self.data_ingestion_artifact.trained_file_path
            test_file_path=self.data_ingestion_artifact.test_file_path

            ## read the data from train and etst 
            train_dataframe=DataValidation.read_data(train_file_path)
            test_dataframe=DataValidation.read_data(test_file_path)

            # validate number of columns 
            status=self.validate_numbers_of_columns(dataframe=train_dataframe)
            if not status:
                error_message=f'{error_message} Train dataframe does not contain all columns \n'
            
            status=self.validate_numbers_of_columns(dataframe=test_dataframe)
            if not status:
                error_message=f'{error_message} Test dataframe does not contain all columns \n'

            ## check for numerical columns
            ## check for data drift 
            
        except Exception as e:
            raise NetworkSecurityException(e,sys)
        
    

